https://www.hackerrank.com/challenges/two-pluses/problem

In [1]:
import os
import csv

In [2]:
dir_path=os.getcwd()
test_path=os.path.join(dir_path,'TwoPluses_testcase03.txt')
sol_path=os.path.join(dir_path,'TwoPluses_testcase03_sol.txt')

In [3]:
with open(test_path, newline='') as csvfile:
    file=csv.reader(csvfile)
    in_list=[]
    for row in file:
        in_list+=row
with open(sol_path, newline='') as csvfile:
    file=csv.reader(csvfile)
    sol_list=[]
    for row in file:
        sol_list+=row
sol_list

['189']

In [4]:
param=in_list[0].strip().split()
n=int(param[0]) # rows
m=int(param[1]) # cols
grid=[]
for i in range(n):
    grid.append([])
    for j in range(m):
        grid[i].append(in_list[i+1][j])

In [5]:
class Vertex:
    def __init__(self, key):
        self.id = key
        self.row = key[0]
        self.col = key[1]
        self.good = ''
        self.connectedTo = []
        self.color=''
        self.plus_size=0
    
    def setPlusSize(self,plus_size):
        self.plus_size=plus_size
    
    def getPlusSize(self):
        return self.plus_size
    
    def setG(self,g):
        self.good=g
        
    def getG(self):
        return self.good
    
    def getRow(self):
        return self.row
    
    def getCol(self):
        return self.col 
        
    def addNeighbor(self,nbr):
        self.connectedTo.append(nbr)
    
    def getConnections(self):
        return self.connectedTo
    
    def getId(self):
        return self.id
    
    def getColor(self):
        return self.color
    
    def setColor(self,newColor):
        self.color=newColor
        
class Graph:
    def __init__(self):
        self.vertList = {}
        self.numVertices = 0
        self.n = 0 
        self.m = 0
        
    def addVertex(self,key):
        self.numVertices = self.numVertices + 1
        newVertex = Vertex(key)
        self.vertList[key] = newVertex
        return newVertex
    
    def getVertices(self):
        return self.vertList.keys()
    
    def setN(self,n):
        self.n=n
    
    def getN(self):
        return self.n
    
    def setM(self,m):
        self.m=m
    
    def getM(self):
        return self.m
    
    def __iter__(self):
        return iter(self.vertList.values())

In [6]:
def searchAround(g,v,cycle):
    n=g.getN()
    m=g.getM()
    i=v.getRow()
    j=v.getCol()
    found=False
    if i-cycle>=0 and j-cycle>=0 and i+cycle<=n-1 and j+cycle<=m-1:
        if g.vertList[(i+cycle,j)].getG()=='G' and g.vertList[(i+cycle,j)].getColor()=='white' and \
            g.vertList[(i-cycle,j)].getG()=='G' and g.vertList[(i-cycle,j)].getColor()=='white' and \
            g.vertList[(i,j+cycle)].getG()=='G' and g.vertList[(i,j+cycle)].getColor()=='white' and \
            g.vertList[(i,j-cycle)].getG()=='G' and g.vertList[(i,j-cycle)].getColor()=='white':
            found=True
    return found

In [7]:
def searchPlus(g,v):
    i=v.row
    j=v.col
    cycle=0
    found=True
    while found:
        cycle+=1
        found=searchAround(g,v,cycle)
    return cycle-1

In [8]:
def twoPluses(grid):
    g=Graph()
    n=len(grid)
    m=len(grid[0])
    g.setN(n)
    g.setM(m)
    
    # make graph
    for i in range(n):
        for j in range(m):
            key=(i,j) # row, col
            newV=g.addVertex(key)
            newV.setG(grid[i][j])
            if newV.getG()=='G':
                newV.setColor('white')
    
    # add neighbors
    for v in g:
        i=v.getRow()
        j=v.getCol()
        if i>0:
            nv=g.vertList[(i-1,j)]
            v.addNeighbor(nv)
        if i<n-1:
            nv=g.vertList[(i+1,j)]
            v.addNeighbor(nv)
        if j>0:
            nv=g.vertList[(i,j-1)]
            v.addNeighbor(nv)
        if j<m-1:
            nv=g.vertList[(i,j+1)]
            v.addNeighbor(nv)
    
    # special case of no Gs
    v_g1=0
    for v in g: # find first G
        if v.getG()=='G':
            v_g1=v.getId()
            break
            
    if not v_g1: # If there are no Gs, return 0.
        return 0
    
    else:
        # add plus_size at each vertex of graph g
        for v in g:
            if v.getG()=='G':
                plus_size=searchPlus(g,v)
                v.setPlusSize(plus_size)
            
        d_ans={}
        list_ans=[]
        
        for v1 in g:  # we have a nested for loop here but 
                      #   since m and n are each only max 15, time should be okay           
            if v1.getG()=='G':
                cycle1=v1.getPlusSize()
                area_1=cycle1*4+1    # area of plus at this v
                
                # set color of each of the G used in this plus as 'grey'
                #   so that they do not overlap with other plus
                v1.setColor('grey')
                i=v1.getRow()
                j=v1.getCol()
                for c in range(cycle1):
                    g.vertList[(i+c,j)].setColor('grey')
                    g.vertList[(i-c,j)].setColor('grey')
                    g.vertList[(i,j+c)].setColor('grey')
                    g.vertList[(i,j-c)].setColor('grey')
                # find size of pluses possible at each G for the second plus
                #   we are only looking for local maximum, so we will not store 
                #   these plus_size to the vertices of g
                cycle2=0
                check=0
                for v2 in g:
                    if v2.getG()=='G' and v2.getColor()=='white':
                        check=1
                        plus_size=searchPlus(g,v2)
                        if plus_size>cycle2:
                            cycle2=plus_size

                area_2=check*(cycle2*4+1) # area of second largest plus
                # if there are no other G, 'check' makes sure we return 0

                # Append the product of areas from this round to list_ans
                list_ans.append(area_1*area_2)
                
                
                d_ans[v1.getId()]=(area_1,area_2)
                
                #prepare for next loop
                v1.setColor('white') # set all G color as white
                
        list_ans.sort(reverse=True)
        print(list_ans)
        return list_ans[0], d_ans

In [10]:
t=twoPluses(grid)
print(t[0])
d_ans=t[1]
print(d_ans[(5,5)])
for i in range(n):
    print(grid[i])

[221, 153, 153, 105, 105, 105, 105, 105, 105, 105, 105, 105, 105, 105, 105, 85, 85, 85, 85, 85, 85, 85, 65, 65, 65, 45, 45, 45, 45, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 17, 17, 9, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
221
(21, 5)
['G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G']
['G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G']
['B', 'G', 'B', 'G', 'G', 'G', 'B', 'G', 'B', 'G', 'B', 'G']
['B', 'G', 'B', 'G', 'G', 'G', 'B', 'G', 'B', 'G', 'B', 'G']
['G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G']
['G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G']
['G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G']
['G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G'